<a href="https://colab.research.google.com/github/nicolashernandez/teaching_nlp/blob/main/LinguisticAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Généralités

### Objectif  
* analyser et interpréter les sorties et la qualité d'analyseurs linguistiques 
* découvrir les caractéristiques (en particulier linguistiques) d'une donnée langagière qui peuvent influer sur les traitements automatiques et la qualité de ceux-ci 

### NLP (python) Libraries
* spaCy: state-of-the-art natural language processing with industrial motivations and tools, multilingue, statistical/neuronal models https://spacy.io/usage/linguistic-features
* NLTK : pédagogique, des approches à base de règles ou statistiques, multilingues
* Stanza : python, nouveau framework de Stanford, modèles neuronaux entraînés sur données UD
* flair, fondé sur PyTorch, multilingue, support spécial pour le biomedical https://github.com/flairNLP/flair
* gensim: topic modeling and similarity detection

Mais aussi en java    
* Stanford Core NLP : multilingue, statistique, résolution de la coréference
* DKPro

### Thèmes abordés

* (Pré-)traitements linguistiques
  * Tokenization, sentence segmentation FIXME
  * POS tagging, morphology, lemmatization, dependency parsing, 
  * Named entities
  * Language detection FIXME
* Mise en application 
  * Langue : Français 
  * Types de texte : Dépèches journalistiques, Twitter, romantic novels and legal text
  * Dimension multilingue

### Consignes de travail

Réponse aux questions dans la section "votre réponse". Des réponses brèves et simples sont attendues. 

Vous avez le droit de modifier le code pour vous permettre de plus facilement répondre aux questions.




# Installation de l'environnement : chargement des modèles et des données

Executer le code suivant

In [ ]:
# Téléchargement d'un modèle pour le traitement du français
!python -m spacy download fr_core_news_sm

In [ ]:
# Importation de la bibliothèque spaCy 
import spacy

# Chargement du modèle pour le français
nlp = spacy.load("fr_core_news_sm")

# Importation d'une liste d'exemple de phrases en français
from spacy.lang.fr.examples import sentences 

Si vous obtenez l'erreur suivante 
```
# OSError: [E050] Can't find model 'fr_core_news_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
```
Alors 
- télécharger le modèle `!python -m spacy download fr_core_news_sm `
- puis Exécution > Redémarrer l'environnement d'Exécution

Executer aussi :


In [ ]:
import nltk

nltk.download('punkt')

# Analyses linguistiques du français 


## Tokénisation

Un **token** est une instance d'une séquence de caractères dans un document donné qui constitue une unité pour une quelconque raison (e.g. délimitée par des espaces). Les tokens qui ont une réalité grammaticale (e.g. Nom, Verbe, Adjectif...) peuvent être considérés comme des instances de **mots**. Le mot a une réalité linguistique (ses caractères sont des lettres de l'alphabet plus l'espace et le tiret). 



### QUESTION

Le code suivant commence par déclarer deux tokenizers utilisant des technologies différentes : l'un est proposé par la bibliothèque _nltk_ et l'autre _spaCy_. Le code se poursuit en calculant le nombre de tokens obtenus par chacun des tokenizers. Puis applique les deux tokenizers et affiche les différences de tokenization pour les 5 1ères phrases du corpus de phrases exemples en français de spaCy.

* Exécuter le code. Quelles erreurs/différences de tokenization observez-vous ?


In [ ]:
# machine learning based model
import spacy
spacy_tokenize = lambda text: [token.text for token in nlp(text)]

# uses regular expressions (to fit the penn treebank corpus tokenization) 
# and an unsupervised sentence segmentation model (aka punkt) 
# to distinguish abreviations from sentence endings.  
from nltk import word_tokenize
nltk_tokenize = lambda text: [token for token in word_tokenize(text, language='french')]

len_nltk_tokens = 0
len_spacy_tokens = 0
for i in range (0, len(sentences)):
  len_nltk_tokens += len(word_tokenize(sentences[i]))
  len_spacy_tokens += len(spacy_tokenize(sentences[i]))
print('len_sentences=',len(sentences),' ; len_nltk_tokenize=', len_nltk_tokens, ' ; len_spacy_tokens=', len_spacy_tokens)
print()

from difflib import context_diff, ndiff
for i in range (0, 5):
  print (i, sentences[i])
  print('\n'.join(context_diff(word_tokenize(sentences[i]), spacy_tokenize(sentences[i]), fromfile='nltk_tokenize', tofile='spacy_tokenize')))
  print()

### VOTRE REPONSE

**TODO**





## Analyse lexicale


### Token vs Mot vs Formes morphologiques vs Lemme

Le **mot** peut avoir plusieurs **formes (morphologiques)** lesquelles renseignent sur le genre, le nombre, le mode/temps (pour les verbes). Le mot est généralement utilisée comme entrée lexicale (dans un disctionnaire). On appelle **lemme** la forme référente d'un mot. "renseigner", "pomme de terre" sont des mots. Ces formes servent aussi de lemmes. "renseignent" est une des formes du mot "renseigner".  


### Propriétés associées à un token mot dans Spacy

Les modèles de Spacy produisent de base une analyse grammaticale, morphologique et syntaxique des mots pour plusieurs langues.
Les propriétés suivantes informent de différents attributs (en particulier linguistiques associées) à chaque token.
* `text`: The original word text
* `lemma_`: The base form of the word.
* `pos_`: The simple UPOS part-of-speech tag.
* `tag_`: The detailed part-of-speech tag with morphological information.
* `dep_`: Syntactic dependency, i.e. the relation between tokens.
* `shape_`: The word shape – capitalization, punctuation, digits.
* `is_alpha`: Is the token an alpha character?
* `is_stop`: Is the token part of a stop list, i.e. the most common words of the language?


### QUESTIONS 
Le code suivant permet d'appliquer un modèle Spacy offrant des traitements TAL à un document (ici une phrase) donné.
* Ajouter les propriétés non spécifiées permettant d'observer les résultats de la lemmatisation, de l'étiquetage grammatical, de l'analyse morphologique et de l'analyse en dépendance syntaxique. 
* Consulter l'analyse des 5 premières phrases exemples de spacy (0 à 4). Trouvez-vous des erreurs de tokenization ? de lemmatisation ? d'étiquetage grammatical ? d'analyse morphologique ? Eventuellement, donner quelques exemples.

In [ ]:
# Ici le document est la 1ère phrase (sentence 0) des exemples de Spacy 
# de phrases écrites en français
doc = sentences[2]

# exécution des traitements en une seule commande
spacy_doc = nlp(doc)

# affichage de la phrase
print(spacy_doc.text)

# affichage de quelques résultats d'analyse 
# et ce, dans une pandas dataframe pour améliorer le visuel
import pandas as pd
spacy_tokens_as_list = [(token.text, token.is_stop) for token in spacy_doc]
pd.DataFrame(spacy_tokens_as_list, columns=['Token', 'is_stop'])

### VOTRE RÉPONSE

**TODO**


In [ ]:
#...

## Analyse syntaxique
En syntaxe, il existe deux modèles d'analyse de la structure syntaxique:
- L'**analyse en constituants** (_chunk_ en anglais) ou **syntagmes** qui met en avant des catégories d'objets syntaxiques : groupe/syntagme nominal, syntagme verbal... avec une structure récursive (e.g. un syntagme verbal peut contenir un syntagme nominal...);
- L'**analyse en dépendances**  qui met en avant les fonctions jouées par des têtes lexicales : sujet, objet, modifieur...

Les constituants peuvent jouer le rôle de termes candidats pour indexer de l'information. On verra que d'autres techniques existent. 
L'arbre syntaxique peut être exploitée en simplification de phrases ou bien dans des systèmes de question-réponse pour identifier la relation entre des entités ou objets.


### QUESTIONS : Analyse en constituants
Le code suivant permet d'observer des constituants nominaux pour les 5 premières phrases des phrases exemples en français de Spacy.

* Les syntagmes identifiés vous semble-t-ils correctes ? Sont ils raccord avec l'analyse grammaticale produite des tokens mots ?

In [ ]:
#spacy_noun_chunks_as_list = [(chunk.text,  chunk.root.text, chunk.root.dep_,
#            chunk.root.head.text) for chunk in spacy_doc.noun_chunks]
#pd.DataFrame(spacy_noun_chunks_as_list, columns=['text', 'root', 'dep', 'head'])
for i in range(0,5):
  spacy_doc = nlp(sentences[i])
  print(i, spacy_doc.text)
  for chunk in spacy_doc.noun_chunks:
    print(chunk.text) #, chunk.root.text, chunk.root.dep_, chunk.root.head.text)   
  print()

### VOTRE RÉPONSE

**TODO**



### QUESTIONS : Analyse en dépendance

Le code suivant permet de visualiser la structure syntaxique en dépendance pour un document (une phrase) donnée.

* Consulter l'analyse des 5 premières phrases exemples de spacy (0 à 4). Si l'on traduit la dépendance comme une forme d'_importance_, les flèches partent du mot le plus important et vont jusqu'aux mots les moins importants dans la phrase. Est-ce bien le cas pour toutes les phrases ? 

In [ ]:
from spacy import displacy

for i in range(0,5):
  spacy_doc = nlp(sentences[i])
  displacy.render(spacy_doc, style='dep', jupyter=True)

### VOTRE RÉPONSE

**TODO**

## Reconnaissance d'entités nommées

### QUESTIONS

Le code suivant permet de visualiser entités nommées présentes dans le document analysé.
* Lister les types d'entités (_labels_) présentes dans les exemples
* Consulter l'analyse des 10 premières phrases exemples de spacy (0 à 9). Trouvez-vous des erreurs de délimitation d'entités nommées ? Dans l'étiquetage du type des entités ? Eventuellement, donner quelques exemples.
* Jeter un oeil sur les performances (section *accuracy evaluation*) des modèles pour le français https://spacy.io/models/fr pour avoir une idée de la performance supposée de ceux-ci.

In [ ]:
# le code suivant permet de visualiser entités nommées présentes dans le document analysé
# les offsets et le type d'entité sont aussi fournis 
for ent in spacy_doc.ents:
   print(ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(doc, style='ent', jupyter=True)

### VOS RÉPONSES

**TODO**


## Expérience personnelle vs performances attestées


### QUESTIONS
* Jeter un oeil sur les performances (section *accuracy evaluation*) du modèle utilisé pour le français https://spacy.io/models/fr pour avoir une idée de la performance supposée de celui-ci sur les tâches d'analyse linguistique (tokenization, étiquetage grammatique, lemmatisation, analyse morphologique, analyse en dépendance et reconnaissance d'entités nommées). Est-ce raccord avec ce que vous avez observé ?


### VOTRE RÉPONSE

**TODO**



# Analyse de textes de genres différents

Le code suivant télécharge dans un répertoire `data` un corpus de phrases issus de 4 genres différents : textes parlementaires européens (_legal europarl_), dépèches journalistiques (_news_wikinews_), littératique romanesque (_roman verne_), et des tweets de twitter (_tweets twitter_).

Exécuter le.


In [ ]:
!mkdir data
!wget -nc https://raw.githubusercontent.com/nicolashernandez/teaching_nlp/main/data/legal_europarl.txt -P data
!wget -nc https://raw.githubusercontent.com/nicolashernandez/teaching_nlp/main/data/news_wikinews.txt -P data
!wget -nc https://raw.githubusercontent.com/nicolashernandez/teaching_nlp/main/data/roman_verne.txt -P data
!wget -nc https://raw.githubusercontent.com/nicolashernandez/teaching_nlp/main/data/tweets_twitter.txt  -P data


## QUESTIONS

Le code suivant charge tour à tour chacun des corpus et traite les 5 premières phrases de chaque corpus. Deux résultats de traitement sont observés : la tokenization et la reconnaissance d'entités nommées à l'aide de spaCy et du modèle pour le français précédemment utilisé.

* Quels problèmes de tokenization relevez-vous suivant les genres de texte ? Y-a-til des genres pour lesquels la tokenization fonctionne mieux que d'autres ?
* Mêmes questions pour la reconnaissance des entités nommées.
* Quels genres de texte ont servi de données d'entraînement pour construire le modèle français utilisé ici avec spaCy (cf. https://spacy.io/models/fr) ? Est-ce que cela peut expliquer les performances observées ?  


In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
filenames = ['legal_europarl', 'news_wikinews', 'roman_verne', 'tweets_twitter']
for filename in filenames:
  with open("data/"+filename+".txt", 'r', encoding='UTF-8') as file:
    i = 0
    print('-->', filename.upper())
    print()
    for line in file:
        doc = line.rstrip()
        spacy_doc = nlp(doc)
        print('Sentence',i,':', doc)
        print ('spacy_tokenize:', [token.text for token in spacy_doc])
        if spacy_doc.ents: displacy.render(spacy_doc, style='ent', jupyter=True)
        #spacy_doc_as_list = [(token.text, token.lemma_, token.pos_) for token in spacy_doc]
#            , token.tag_,token.shape_, token.is_alpha, token.is_stop) for token in doc]
        #print (pd.DataFrame(spacy_doc_as_list, columns=['Token', 'Lemma', 'POS']))
        print ()
        i += 1
        if i==5: break
    print ('------------------------------------------------------------------')


## VOTRE RÉPONSE

**TODO**


# Biais des modèles construits sur des données

Soit le texte suivant
> *Elle est médecin. Il est infirmier.*



### QUESTIONS
* Ouvrir [Google Translate dans votre navigateur](https://translate.google.fr/?hl=fr&sl=fr&tl=en&text=Elle%20est%20m%C3%A9decin.%20Il%20est%20infirmier.&op=translate)
* Traduire du français (langue source) vers l'anglais (langue cible). Cliquer deux fois sur "Intervertir les langues" (pour traduire une fois vers l'anglais puis pour retraduire l'anglais vers le français). Observez-vous quelque chose ?
* Faire la même chose en prenant comme langue cible du Hongrois. Observez vous quelque chose ? 

### VOTRE REPONSE

**TODO**



---
# Multilinguisme




Donnés les éléments suivants :
* https://spacy.io/models
* https://fr.wikipedia.org/wiki/Liste_de_langues_par_nombre_total_de_locuteurs
* Le score F-score est une moyenne des scores de Précision et de Rappel. 
* un tableau des performances de spaCy telles que présentées en oct 2021 pour le 1er modèle de chaque langue



## QUESTIONS 
* Peut-on considérer que la _tokenization_ est un problème réglé ? Quelle est la langue pour laquelle la performance est la plus basse ? Selon vous, est-ce un problème de ressources ou bien de spécificité de la langue ? 
oui, chinois, spécificité de la langue
* Peut-on considérer que la _segmentation en phrase_ est un problème réglé ? Quelles sont les deux langues pour lesquelles les performances sont les plus basses ? Selon vous, est-ce un problème de ressources ou bien de spécificité de la langue ?  
chinois et macédonien ; resp. spécificité de la langue et probablement manque de ressource
* Pourquoi des performances d'analyse morphologique et grammatical ne sont pas rapportées pour toutes les langues ?
des langues n'ont pas flexions morphorlogiques
* Peut-on considérer que l' _étiquetage grammatical_ est un problème réglé ? 
globalement oui
* Peut-on considérer que l' _analyse morphologique comprenant la lemmatisation_ est un problème réglé ?
non pour certaines langues 
non pour les langues asiatiques et moins bien dotées que pour les autres
* Peut-on considérer que la _reconnaissance des entités nommées_ est un problème réglé ? 
non, globalement encore 1.5 de progression, difficultés inhérentes aux langues asiatiques et faible résultat des langues peu dotées
* Peut-on considérer que l'_analyse syntaxique en dépendance_ est un problème réglé ? 
même constat
* Citer deux langues parmi les langues les plus parlées dans le monde (en nombre total de locuteur) qui ne sont pas prises en charge par spacy.
hindi, arabe
* Trouvez-vous sur le web des bibliothèques (python) qui offrent (partiellement ou totalement) les mêmes traitements que Spacy sur ces langues ? 



## VOS REPONSES

**TODO**

# Benchmark
* https://spacy.io/usage/facts-figures



## QUESTIONS
* En observant les performances (qualité et temps de traitement) de solutions alternatives (Stanza, Flair) sur des tâches reconnaissance d'entités nommées et de parsing en anglais, diriez-vous que Spacy est 1) la meilleure solution, 2) une solution état de l'art, 3) de performance moindre que les solutions existantes ?



## VOTRE REPONSE

**TODO**
